In [67]:
import tensorflow as tf
import pandas as pd
import calendar
import time
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, Imputer

In [7]:
books = pd.read_csv(r"../Dataset/Final/BX-Books.csv")
ratings = pd.read_csv(r"../Dataset/Final/BX-Book-Ratings.csv")
users = pd.read_csv(r"../Dataset/Final/BX-Users.csv")

/usr/local/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [9]:
users.head()

,User-ID,Age,City,State,Country
0,1,20.0,Airmont,New York,United States
1,2,18.0,Stockton,California,United States
2,3,21.0,Moscow,Moscow,Russia
3,4,17.0,Porto,Norte,Portugal
4,5,23.0,Farnborough,England,United Kingdom


In [10]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,g_year-of-publication,g_categories,g_processed
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2003,Social Science,1
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,2001,Actresses,1
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,1983,1940-1949,1
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,1999,Medical,1
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,1999,Design,1


In [11]:
full_data_df = ratings.merge(books,on='ISBN').merge(users,on='User-ID')

In [12]:
full_data_df.count()

User-ID                  1031173
ISBN                     1031173
Book-Rating              1031173
Book-Title               1031173
Book-Author              1031172
Year-Of-Publication      1031173
Publisher                1031171
Image-URL-S              1031173
Image-URL-M              1031173
Image-URL-L              1031173
g_year-of-publication    1031173
g_categories             1031173
g_processed              1031173
Age                      1031173
City                     1031173
State                    1031173
Country                  1031173
dtype: int64

In [18]:
useful_data_df = full_data_df[['City', 'State', 'Country', 'Age', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Book-Rating']]
useful_data_df.head()

,City,State,Country,Age,Book-Author,Year-Of-Publication,Publisher,Book-Rating
0,Tyler,Texas,United States,21.0,M. J. Rose,2002,Ballantine Books,0
1,Cincinnati,Ohio,United States,23.0,M. J. Rose,2002,Ballantine Books,5
2,Cincinnati,Ohio,United States,23.0,Orson Scott Card,1986,Tor Books,9
3,Cincinnati,Ohio,United States,23.0,TRUMAN CAPOTE,1994,Vintage,8
4,Cincinnati,Ohio,United States,23.0,Rebecca Wells,1996,HarperCollins,9


In [14]:
def getLabelEncodedColumn(col, df):
    le = LabelEncoder()
    return le.fit_transform(df[col].values).reshape(-1,1)

In [19]:
useful_data_df = useful_data_df.dropna()

In [20]:
useful_data_df.count()

City                   1031170
State                  1031170
Country                1031170
Age                    1031170
Book-Author            1031170
Year-Of-Publication    1031170
Publisher              1031170
Book-Rating            1031170
dtype: int64

In [21]:
useful_data_df['Book-Author-Enc'] = getLabelEncodedColumn('Book-Author', useful_data_df)
useful_data_df['City-Enc'] = getLabelEncodedColumn('City', useful_data_df)
useful_data_df['State-Enc'] = getLabelEncodedColumn('State', useful_data_df)
useful_data_df['Country-Enc'] = getLabelEncodedColumn('Country', useful_data_df)
useful_data_df['Publisher-Enc'] = getLabelEncodedColumn('Publisher', useful_data_df)

In [22]:
useful_data_df.head()

,City,State,Country,Age,Book-Author,Year-Of-Publication,Publisher,Book-Rating,Book-Author-Enc,City-Enc,State-Enc,Country-Enc,Publisher-Enc
0,Tyler,Texas,United States,21.0,M. J. Rose,2002,Ballantine Books,0,61693,6860,927,157,1391
1,Cincinnati,Ohio,United States,23.0,M. J. Rose,2002,Ballantine Books,5,61693,1526,686,157,1391
2,Cincinnati,Ohio,United States,23.0,Orson Scott Card,1986,Tor Books,9,73674,1526,686,157,14901
3,Cincinnati,Ohio,United States,23.0,TRUMAN CAPOTE,1994,Vintage,8,93471,1526,686,157,15650
4,Cincinnati,Ohio,United States,23.0,Rebecca Wells,1996,HarperCollins,9,80510,1526,686,157,6614


In [29]:
useful_df = useful_data_df.filter(['Age', 'Year-Of-Publication','Publisher-Enc',
                                   'Book-Author-Enc', 'City-Enc','State-Enc', 'Book-Rating'])

In [30]:
useful_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031170 entries, 0 to 1031172
Data columns (total 7 columns):
Age                    1031170 non-null float64
Year-Of-Publication    1031170 non-null float64
Publisher-Enc          1031170 non-null float64
Book-Author-Enc        1031170 non-null float64
City-Enc               1031170 non-null float64
State-Enc              1031170 non-null float64
Book-Rating            1031170 non-null float64
dtypes: float64(7)
memory usage: 62.9 MB


In [31]:
X, y = useful_df.drop('Book-Rating', axis=1), useful_data_df[['Book-Rating']]

In [32]:
X.head()

,Age,Year-Of-Publication,Publisher-Enc,Book-Author-Enc,City-Enc,State-Enc
0,21.0,2002.0,1391.0,61693.0,6860.0,927.0
1,23.0,2002.0,1391.0,61693.0,1526.0,686.0
2,23.0,1986.0,14901.0,73674.0,1526.0,686.0
3,23.0,1994.0,15650.0,93471.0,1526.0,686.0
4,23.0,1996.0,6614.0,80510.0,1526.0,686.0


In [33]:
y.head()

,Book-Rating
0,0.0
1,5.0
2,9.0
3,8.0
4,9.0


In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [220]:
RUN_NAME = calendar.timegm(time.gmtime())

learning_rate = 0.001
training_epochs = 30

number_of_inputs = 5
number_of_outputs = 1

layer_1_nodes = 5
layer_2_nodes = 25
layer_3_nodes = 12

In [36]:
from sklearn.preprocessing import OneHotEncoder
def getOneHotEncoding(column):
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit_transform(useful_data_df[column].values.reshape(-1,  1))
    return enc

In [37]:
def oneHotCodedSparseMatrix():
    return {
        'author': getOneHotEncoding('Book-Author-Enc'),
        'city': getOneHotEncoding('City-Enc'),
        'state': getOneHotEncoding('State-Enc')
    }


In [38]:
def embeddingsInputLen():
    ohcsm = oneHotCodedSparseMatrix()
    return {
        'author': len(ohcsm['author'].active_features_),
        'city': len(ohcsm['city'].active_features_),
        'state': len(ohcsm['state'].active_features_)
    }

In [39]:
embeddingsLength = embeddingsInputLen()
embeddingsLength

{'author': 101590, 'city': 7574, 'state': 1065}

In [142]:
def getInputLayers(name):
    return tf.placeholder(tf.float64, shape=(1, 1), name=name)

### Input Layer: Age, City, State, Author, Year of Publish

In [169]:
tf.reset_default_graph()
input_age = getInputLayers('input_age')
input_state = getInputLayers('input_state') 
input_author = getInputLayers('input_author')
input_city = getInputLayers('input_city')
input_year = getInputLayers('input_year')

### Embeddings: City, State, Author

In [170]:
weights_city = tf.get_variable(name="city_emb_wt", shape=[embeddingsLength['city'], 2], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
encoded_city = tf.one_hot(tf.cast(input_city[0], tf.int64), embeddingsLength['city'])
embedding_city_output = tf.nn.relu(tf.matmul(tf.cast(encoded_city, tf.float64), weights_city))

In [171]:
weights_state = tf.get_variable(name="state_emb_wt", shape=[embeddingsLength['state'], 2], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
encoded_state = tf.one_hot(tf.cast(input_state[0], tf.int64), embeddingsLength['state'])
embedding_state_output = tf.nn.relu(tf.matmul(tf.cast(encoded_state, tf.float64), weights_state))

In [172]:
weights_author = tf.get_variable(name="author_emb_wt", shape=[embeddingsLength['author'], 2], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
encoded_author = tf.one_hot(tf.cast(input_author[0], tf.int64), embeddingsLength['author'])
embedding_author_output = tf.nn.relu(tf.matmul(tf.cast(encoded_author, tf.float64), weights_author))

### Layer 1

In [173]:
input_x = tf.concat([input_age, embedding_city_output, embedding_state_output, embedding_author_output, input_year], axis=1)
weights_1 = tf.get_variable(name="weights1", shape=[8, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
biases_1 = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer(), dtype=tf.float64)
layer_1_output = tf.nn.relu(tf.matmul(input_x, weights_1) + biases_1)

### Layer 2

In [174]:
weights_2 = tf.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
biases_2 = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer(), dtype=tf.float64)
layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights_2) + biases_2)

### Layer 3

In [175]:
weights_3 = tf.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
biases_3 = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer(), dtype=tf.float64)
layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights_3) + biases_3)

### Output

In [176]:
weights_o = tf.get_variable("weights4", shape=[layer_3_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
biases_o = tf.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer(), dtype=tf.float64)
prediction = tf.matmul(layer_3_output, weights_o) + biases_o

### Cost

In [177]:
actual_y = tf.placeholder(tf.float64, shape=(None, 1), name="Actual_Y")
cost = tf.reduce_mean(tf.squared_difference(prediction, actual_y))

### Optimizer

In [178]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### Logging

In [179]:
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

In [216]:
y.columns

Index(['Book-Rating'], dtype='object')

In [217]:
def getFeedDict(idx):
    try:
        age = X.loc[idx,'Age'].reshape(1,1)
        yop = X.loc[idx,'Year-Of-Publication'].reshape(1,1)
        pub = X.loc[idx,'Publisher-Enc'].reshape(1,1)
        atr = X.loc[idx,'Book-Author-Enc'].reshape(1,1)
        city = X.loc[idx,'City-Enc'].reshape(1,1)
        state = X.loc[idx,'State-Enc'].reshape(1,1)
        output = y.loc[idx,'Book-Rating'].reshape(1,1)
    except:
        print(idx)
        raise IndexError("single positional indexer is out-of-bounds") 
    return {input_age: age, input_year: yop, input_author: atr, input_city: city, input_state: state, actual_y: output}

### Session Initialization

In [219]:
%%time
with tf.Session() as session:
    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    training_writer = tf.summary.FileWriter("./logs/{}/training".format(RUN_NAME), session.graph)
    testing_writer = tf.summary.FileWriter("./logs/{}/testing".format(RUN_NAME), session.graph)

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())
    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(1):
        # Feed in the training data and do one step of neural network training
        # 'Age', 'Year-Of-Publication', 'Book-Author-Enc', 'City-Enc','State-Enc'
        for row in X_train.itertuples():
            feedDict = getFeedDict(row.Index)
            session.run(optimizer, feed_dict=feedDict)
        '''
        if epoch % 5 == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            for row in X_train.itertuples():
                feedDict = getFeedDict(row.Index)
                training_cost, training_summary = session.run([cost, summary], feed_dict=feedDict)
            for row in X_test.itertuples():
                feedDict = getFeedDict(row.Index)
                testing_cost, training_summary = session.run([cost, summary], feed_dict=feedDict)
            # Write the current training status to the log files (Which we can view with TensorBoard)
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)
            # Print the current training status to the screen
            print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))
        '''
    for row in X_test.itertuples():
        feedDict = getFeedDict(row.Index)
        testing_cost, training_summary = session.run([cost, summary], feed_dict=feedDict)
    print("Testing Cost: {}".format(testing_cost))

        

Testing Cost: 7.799451156407845


In [ ]:
%%time
train_cost = {};
test_cost = {};
with tf.Session() as session:
    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    training_writer = tf.summary.FileWriter("./logs/{}/training".format(RUN_NAME), session.graph)
    testing_writer = tf.summary.FileWriter("./logs/{}/testing".format(RUN_NAME), session.graph)

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())
    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):
        # Feed in the training data and do one step of neural network training
        # 'Age', 'Year-Of-Publication', 'Book-Author-Enc', 'City-Enc','State-Enc'
        for row in X_train.itertuples():
            feedDict = getFeedDict(row.Index)
            session.run(optimizer, feed_dict=feedDict)
        if epoch % 5 == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            for row in X_train.itertuples():
                feedDict = getFeedDict(row.Index)
                training_cost, training_summary = session.run([cost, summary], feed_dict=feedDict)
            for row in X_test.itertuples():
                feedDict = getFeedDict(row.Index)
                testing_cost, testing_summary = session.run([cost, summary], feed_dict=feedDict)
            # Write the current training status to the log files (Which we can view with TensorBoard)
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)
            # Print the current training status to the screen
            train_cost[epoch] = training_cost;
            test_cost[epoch] = testing_cost;
            print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))
    for row in X_test.itertuples():
        feedDict = getFeedDict(row.Index)
        testing_cost, training_summary = session.run([cost, summary], feed_dict=feedDict)
    print("Testing Cost: {}".format(testing_cost))

        

In [32]:
! tensorboard --logdir "./logs"

/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
TensorBoard 0.4.0 at http://domino-run-5e98554246e0fb000c9a3010:6006 (Press CTRL+C to quit)
W0416 13:27:50.828567 Reloader tf_logging.py:86] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0416 13:27:50.829031 Reloader tf_logging.py:86] Found more than one metagraph event per run. Overwriting the metagraph with the newest event.
W0416 13:27:50.837380 Reloader tf_logging.py:86] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0416 13:27:50.837770 Reloader tf_logging.py:86] Found more than one metagraph event per run. Overwr

In [85]:
import numpy 
numpy.array([1.2, "abc"], dtype=object)

array([1.2, 'abc'], dtype=object)